In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

In [7]:
from transformers import BlipForConditionalGeneration, BlipProcessor
import torch
import torch.nn as nn  # Make sure to import nn module
import torch.optim as optim
import numpy as np

# Load BLIP model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")


In [30]:
import torch
import numpy as np
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load your model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to('cuda')

# Load your EEG embeddings
mapped_embeddings = np.load('/kaggle/input/modified-embeddings/modified_eeg_embeddings.npy')  # Load the preprocessed EEG embeddings
mapped_embeddings = torch.tensor(mapped_embeddings).float().to('cuda')

# Set batch size for processing
batch_size = 32

# Check feature dimensions and prepare reshaping
batch_size, feature_dim = mapped_embeddings.shape

# Calculate height and width dynamically based on feature_dim (without assuming a square shape)
height = None
width = None

# Loop through possible height values and calculate width
for h in range(1, int(feature_dim**0.5) + 1):
    if feature_dim % h == 0:
        height = h
        width = feature_dim // h
        break

# If we can't find a valid height/width pair, raise an error
if height is None or width is None:
    raise ValueError(f"Feature dimension ({feature_dim}) cannot be reshaped into valid height and width.")

# Reshape mapped EEG embeddings to 4D tensor (batch_size, 1, height, width)
reshaped_embeddings = mapped_embeddings.view(batch_size, 1, height, width)

# Repeat the single channel into 3 channels to mimic RGB-like input (required by the vision model)
reshaped_embeddings = reshaped_embeddings.repeat(1, 3, 1, 1)  # Convert [batch_size, 1, height, width] to [batch_size, 3, height, width]

# Pad the input to make sure it is large enough for the convolutional layers
# For example, padding to (32, 32) or larger if necessary
padding = (0, max(0, 32 - width), 0, max(0, 32 - height))  # Pad the width and height to 32 if necessary
reshaped_embeddings = torch.nn.functional.pad(reshaped_embeddings, padding)

# Generate text descriptions with progress bar
generated_texts = []
with tqdm(total=len(mapped_embeddings), desc="Generating Text from EEG Embeddings") as progress_bar:
    for i in range(0, len(mapped_embeddings), batch_size):
        batch = reshaped_embeddings[i:i + batch_size]  # Slice the batch

        # Ensure the batch has the correct shape for the vision model (batch_size, 3, height, width)
        pixel_values = batch  # Now the batch is in the expected 4D shape [batch_size, 3, height, width]
        
        # Generate text using the model
        outputs = model.generate(pixel_values=pixel_values)  # Pass pixel values to generate text
        
        # Decode and store the generated text
        for output in outputs:
            generated_texts.append(model.decode(output, skip_special_tokens=True))

        progress_bar.update(batch_size)  # Update progress bar

# Save generated texts to a file
np.save('eeg_generated_texts_blip.npy', generated_texts)
print("Generated texts saved!")


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.03 GiB. GPU 0 has a total capacity of 14.74 GiB of which 616.12 MiB is free. Process 2562 has 14.14 GiB memory in use. Of the allocated memory 13.68 GiB is allocated by PyTorch, and 344.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
################################### Using a different model for EEG to text conversion ###################################################

In [8]:
## EEG to Text Embedding Mapping

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load EEG and text embeddings
eeg_embeddings = np.load('/kaggle/input/modified-embeddings/modified_eeg_embeddings.npy')
text_embeddings = np.load('/kaggle/input/modified-embeddings/modified_text_embeddings.npy')

# Convert to PyTorch tensors
eeg_embeddings = torch.tensor(eeg_embeddings, dtype=torch.float32).to(device)
text_embeddings = torch.tensor(text_embeddings, dtype=torch.float32).to(device)

# Dataset and DataLoader
dataset = TensorDataset(eeg_embeddings, text_embeddings)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Define Mapping Network
class EEGToTextMapper(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EEGToTextMapper, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )
    
    def forward(self, eeg_embedding):
        return self.network(eeg_embedding)

# Model and Optimizer
input_dim = eeg_embeddings.shape[1]
output_dim = text_embeddings.shape[1]
model = EEGToTextMapper(input_dim, output_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Training Loop
epochs = 10
model.train()
for epoch in range(epochs):
    epoch_loss = 0
    with tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}") as progress_bar:
        for eeg_batch, text_batch in progress_bar:
            optimizer.zero_grad()
            predicted_text_embedding = model(eeg_batch)
            loss = criterion(predicted_text_embedding, text_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            progress_bar.set_postfix({"Batch Loss": loss.item()})
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {epoch_loss/len(dataloader):.4f}")

# Save the mapping model
torch.save(model.state_dict(), 'eeg_to_text_mapping_model.pth')


Using device: cuda


Epoch 1/10: 100%|██████████| 259/259 [00:00<00:00, 284.41it/s, Batch Loss=0.00084] 


Epoch 1/10, Average Loss: 0.0008


Epoch 2/10: 100%|██████████| 259/259 [00:00<00:00, 289.89it/s, Batch Loss=0.000756]


Epoch 2/10, Average Loss: 0.0008


...

Epoch 8/10, Average Loss: 0.0007


Epoch 9/10: 100%|██████████| 259/259 [00:00<00:00, 287.54it/s, Batch Loss=0.00078] 


Epoch 9/10, Average Loss: 0.0007


Epoch 10/10: 100%|██████████| 259/259 [00:00<00:00, 285.26it/s, Batch Loss=0.000679]


Epoch 10/10, Average Loss: 0.0007


In [10]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained text decoder (GPT-2)
text_decoder = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
text_decoder.config.pad_token_id = text_decoder.config.eos_token_id

# Ensure EEG embeddings are correctly loaded and processed
eeg_embeddings = np.load('/kaggle/input/modified-embeddings/modified_eeg_embeddings.npy')
eeg_embeddings = torch.tensor(eeg_embeddings, dtype=torch.float32).to(device)

# Load trained mapping model
model.load_state_dict(torch.load('/kaggle/working/eeg_to_text_mapping_model.pth', map_location=device))  # Load trained weights
model.to(device)
model.eval()

# Generate mapped embeddings
mapped_embeddings = []
with torch.no_grad():
    for eeg_batch in DataLoader(TensorDataset(eeg_embeddings), batch_size=64):
        eeg_batch = eeg_batch[0]
        mapped_embeddings.append(model(eeg_batch.to(device)))

mapped_embeddings = torch.cat(mapped_embeddings, dim=0)

# Generate text for each mapped embedding with improved visualization
os.makedirs("generated_texts_logs", exist_ok=True)
generated_texts = []

print("Starting text generation...")
with tqdm(total=len(mapped_embeddings), desc="Processing EEG to Text", unit="emb") as pbar:
    for i, embedding in enumerate(mapped_embeddings):
        input_ids = tokenizer.encode("<|startoftext|>", return_tensors="pt").to(device)
        attention_mask = torch.ones_like(input_ids)  # Explicit attention mask

        # Generate text
        outputs = text_decoder.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=50,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            temperature=0.7,
            do_sample=True
        )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_texts.append(text)

        # Add dynamic information to the progress bar
        pbar.set_postfix({
            "Current Index": i,
            "Generated Text Snippet": text[:30],  # Display the first 30 characters
        })

        # Save intermediate results every 100 steps
        if (i + 1) % 100 == 0:
            np.save(f"generated_texts_logs/intermediate_texts_{i + 1}.npy", generated_texts)

        pbar.update(1)

# Save final results
np.save('generated_texts_from_eeg.npy', generated_texts)
print("Generated texts saved!")


Using device: cuda


/tmp/ipykernel_30/3239209276.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/eeg_to_text_mapping_model.pth', map_locat

Starting text generation...


Processing EEG to Text:   0%|          | 0/16540 [00:00<?, ?emb/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing EEG to Text:   0%|          | 0/16540 [00:00<?, ?emb/s, Current Index=0, Generated Text Snippet=<|startoftext|>|h1|

Processing EEG to Text:   0%|          | 1/16540 [00:00<2:00:04,  2.30emb/s, Current Index=0, Generated Text Snippet=<|startoftext|>|h1|

<textarea]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing EEG to Text:   0%|          | 2/16540 [00:00<1:54:42,  2.40emb/s, Current Index=1, Generated Text Snippet=<|startoftext|>|<{left:0}> - <]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing EEG to Text:   0%|          | 2/16540 [00:01<1:54:42,  2.40emb/s, Current Index=2, Generated Text Snippet=<|startoftext|>

Processing EEG to Text:   0%|          | 3/16540 [00:01<1:53:02,  2.44emb/s, Current Index=2, Generated Text Snippet=<|startoftext|>

<text> <count]Setting `pad_to

KeyboardInterrupt: 

In [12]:
import numpy as np

# Load the data
generated_texts = np.load('/kaggle/input/texts/generated_texts_from_eeg.npy', allow_pickle=True)
generated_captions = np.load('/kaggle/input/texts/generated_captions.npy', allow_pickle=True)

print(generated_texts)
print()
print(generated_captions)

# Check alignment
assert len(generated_texts) == len(generated_captions), "Mismatch in dataset lengths!"


['<|startoftext|>|replace|</b>\\n\n\n</div>\n.\n I want to check if the following is valid:\n: (startOftext) {\n,\n (replace)|>{},'
 '<|startoftext|>|<\n\n<[tag=`/|-&-][ \\\\ ]>& \\\\ / | | \\| \\>/g |> \\-| \\x1| \\\\ + \\\\ - \\\\ |'
 '<|startoftext|> | |<</start>| | <end>12|\n\n|<textarea>\n\n\nThis is the code where I put the line.\n "|" is a textarea inside the <'
 ...
 '<|startoftext|>|\\u003cp\\r\n\n\\t\\tset \\tstretch\\tif( \\\n,|\\\\r - $:/core/wordwrap/match.txt \\)) { \\tt'
 '<|startoftext|>|<input type="text" name="name" value="description" /> <input name="" name="description">Description of the entry</input> <property name=\'description\' value=\'Description\' />'
 '<|startoftext|> </head> <body> <!-- this is exactly what we want--> <link href="http://www.mediafire.com/?0WZBxgYJwCq8dLQp']

['a small pig walking across a dirt covered ground'
 'a small pig walking across a dirt covered ground'
 'a small deer is sitting in a pile of hay' ...
 'a box full of zus and zus' 'thr

In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.3 MB/s eta 0:00:00:00:01


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load a sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
captions_embeddings = model.encode(generated_captions)
texts_embeddings = model.encode(generated_texts)

# Calculate cosine similarity
similarities = cosine_similarity(captions_embeddings, texts_embeddings)
average_similarity = similarities.diagonal().mean()
print(f"Average similarity: {average_similarity:.4f}")


Batches:   0%|          | 0/517 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Average similarity: 0.0430


In [13]:
#################################################### TRIAL 202 ##############################################################################

In [16]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm

# Set device (using CUDA if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load EEG and text embeddings
eeg_embeddings = np.load('/kaggle/input/modified-embeddings/modified_eeg_embeddings.npy')  # Replace with correct path
text_embeddings = np.load('/kaggle/input/modified-embeddings/modified_text_embeddings.npy')  # Replace with correct path

# Convert EEG and text embeddings to PyTorch tensors
eeg_embeddings = torch.tensor(eeg_embeddings, dtype=torch.float32).to(device)
text_embeddings = torch.tensor(text_embeddings, dtype=torch.float32).to(device)

# Define Mapping Network (to map EEG to Text Embeddings)
class EEGToTextMapper(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(EEGToTextMapper, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim)
        )
    
    def forward(self, eeg_embedding):
        return self.network(eeg_embedding)

# Create a model for mapping EEG to text embeddings
input_dim = eeg_embeddings.shape[1]  # Dimension of EEG embedding
output_dim = text_embeddings.shape[1]  # Dimension of text embedding
model = EEGToTextMapper(input_dim, output_dim).to(device)

# Optimizer and Loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Training loop (Mapping EEG to Text Embeddings)
epochs = 10
for epoch in range(epochs):
    epoch_loss = 0
    model.train()
    for eeg_batch, text_batch in DataLoader(TensorDataset(eeg_embeddings, text_embeddings), batch_size=64, shuffle=True):
        optimizer.zero_grad()
        predicted_text_embedding = model(eeg_batch)
        loss = criterion(predicted_text_embedding, text_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(DataLoader(TensorDataset(eeg_embeddings, text_embeddings), batch_size=64)):.4f}")

# Save the trained model
torch.save(model.state_dict(), 'eeg_to_text_mapping_model.pth')

# Load GPT-2 for text generation
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set pad_token_id to eos_token_id for GPT-2
gpt2_model.config.pad_token_id = gpt2_model.config.eos_token_id

# Load the trained EEG-to-text mapping model
model.load_state_dict(torch.load('eeg_to_text_mapping_model.pth'))
model.eval()

# Function to convert mapped text embeddings to human-readable text using GPT-2
def generate_text_from_mapped_embeddings(mapped_embeddings):
    generated_texts = []
    with torch.no_grad():
        for i, embedding in enumerate(tqdm(mapped_embeddings, desc="Generating Text from EEG Embeddings")):
            input_ids = tokenizer.encode("<|startoftext|>", return_tensors="pt").to(device)
            attention_mask = torch.ones_like(input_ids).to(device)

            # Generate text using GPT-2
            outputs = gpt2_model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=50,
                num_return_sequences=1,
                no_repeat_ngram_size=2,
                temperature=0.7,
                do_sample=True
            )

            text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            generated_texts.append(text)

            # Print the generated text every 10 iterations
            if (i + 1) % 10 == 0:
                print(f"Generated Text at iteration {i + 1}: {text[:50]}")  # Show first 50 characters

    return generated_texts

# Map EEG embeddings to text embeddings
mapped_embeddings = []
with torch.no_grad():
    for eeg_batch in DataLoader(TensorDataset(eeg_embeddings), batch_size=64):
        eeg_batch = eeg_batch[0].to(device)
        mapped_embeddings.append(model(eeg_batch))

# Concatenate the mapped embeddings
mapped_embeddings = torch.cat(mapped_embeddings, dim=0)

# Generate text from the mapped embeddings
generated_texts = generate_text_from_mapped_embeddings(mapped_embeddings)

# Save the generated texts
np.save('generated_texts_from_eeg.npy', generated_texts)
print("Generated texts saved!")


Using device: cuda
Epoch 1/10, Loss: 0.0008
Epoch 2/10, Loss: 0.0008
Epoch 3/10, Loss: 0.0008
Epoch 4/10, Loss: 0.0008
Epoch 5/10, Loss: 0.0007
Epoch 6/10, Loss: 0.0007
Epoch 7/10, Loss: 0.0007
Epoch 8/10, Loss: 0.0007
Epoch 9/10, Loss: 0.0007
Epoch 10/10, Loss: 0.0007


/tmp/ipykernel_30/2240885403.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('eeg_to_text_mapping_model.pth'))
Generating Text from EEG

Generated Text at iteration 10: <|startoftext|>|<textarea> ||</textbox> <text/>

<


Generating Text from EEG Embeddings:   0%|          | 20/16540 [00:08<1:51:09,  2.48it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text at iteration 20: <|startoftext|>

| endofline|
 (
[{|c-}a]
] |
-n 1


...

Generated Text at iteration 190: <|startoftext|>=.<*></start> <endofline>$1</endlin


Generating Text from EEG Embeddings:   1%|          | 200/16540 [01:19<1:50:15,  2.47it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Text at iteration 200: <|startoftext|>

</style> </head> <body> { "title"


Generating Text from EEG Embeddings:   1%|          | 206/16540 [01:22<1:48:52,  2.50it/s]


KeyboardInterrupt: 

In [27]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize DataParallel if multiple GPUs are available
if torch.cuda.device_count() > 1:
    print("Using 2 GPUs!")
    # Wrap the model with DataParallel to utilize both GPUs
    blip_model = torch.nn.DataParallel(BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")).to(device)
else:
    # Use single GPU or CPU
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# Load the BLIP processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# Load EEG embeddings (Replace with your actual path)
eeg_embeddings = np.load('/kaggle/input/modified-embeddings/modified_eeg_embeddings.npy')
eeg_embeddings = torch.tensor(eeg_embeddings, dtype=torch.float32).to(device)

# Normalize EEG data to the range [0, 1] if not already done
eeg_embeddings = (eeg_embeddings - eeg_embeddings.min()) / (eeg_embeddings.max() - eeg_embeddings.min())

# Reshape EEG embeddings to (batch_size, 1, 1, num_features) for BLIP input
eeg_embeddings = eeg_embeddings.unsqueeze(1).unsqueeze(2)  # Adding 2 singleton dimensions for image-like data

# Duplicate across 3 channels to simulate RGB images
eeg_embeddings_3channel = eeg_embeddings.repeat(1, 3, 1, 1)  # (batch_size, 3, height, width)

# DataLoader to process EEG data in batches
dataset = TensorDataset(eeg_embeddings_3channel)
dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

# List to store generated captions
generated_texts = []

# Generate text from EEG embeddings using BLIP
print("Starting text generation using BLIP...")
with tqdm(total=len(eeg_embeddings), desc="Processing EEG to Text", unit="batch") as pbar:
    for i, batch in enumerate(dataloader):
        eeg_batch = batch[0].to(device)

        # Use BLIP to generate captions (text)
        inputs = processor(images=eeg_batch, return_tensors="pt", do_rescale=False).to(device)  # Process EEG data as image input
        
        # Use blip_model.module if DataParallel is used
        output = blip_model.module.generate(**inputs, max_length=50, num_beams=5, temperature=1.0)  # Generate captions

        # Decode and store generated text
        caption = processor.decode(output[0], skip_special_tokens=True)
        generated_texts.append(caption)

        # Display the caption at every 10 iterations
        if (i + 1) % 10 == 0:
            print(f"Generated Text at iteration {i + 1}: {caption[:50]}...")  # Show first 50 characters

        pbar.update(1)

# Save generated captions
np.save("generated_texts_from_eeg.npy", generated_texts)
print("Generated texts saved!")


Using device: cuda
Using 2 GPUs!
Starting text generation using BLIP...


Processing EEG to Text:   0%|          | 10/16540 [00:34<15:47:19,  3.44s/batch]

Generated Text at iteration 10: a black and white photo of a man in a suit and tie...


Processing EEG to Text:   0%|          | 20/16540 [01:09<16:36:46,  3.62s/batch]

Generated Text at iteration 20: a black and white photo of a man in a suit and tie...


...

Generated Text at iteration 30: a black and white photo of a man in a suit and tie...


Processing EEG to Text:   0%|          | 40/16540 [02:23<16:33:53,  3.61s/batch]

Generated Text at iteration 40: a black and white photo of a man in a suit and tie...


Processing EEG to Text:   0%|          | 45/16540 [02:42<16:33:37,  3.61s/batch]


KeyboardInterrupt: 